In [1]:
import pandas as pd
import numpy as np
import math
import re
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
from datetime import datetime

In [2]:
"""
Récupération des données

temps pour 1 dataset : 1 minute
"""
print(datetime.now().time())

# Récupération des données des combined_data_X.txt

df_1 = pd.read_csv('./data/combined_data_1.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])
# df_2 = pd.read_csv('./data/combined_data_2.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])
# df_3 = pd.read_csv('./data/combined_data_3.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])
# df_4 = pd.read_csv('./data/combined_data_4.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])


# change de le champ "Rating" en float

df_1['Rating'] = df_1['Rating'].astype(float)
# df_2['Rating'] = df_2['Rating'].astype(float)
# df_3['Rating'] = df_3['Rating'].astype(float)
# df_4['Rating'] = df_4['Rating'].astype(float)


# Formation du dataset final

df = df_1
# df = df_1.append(df_2)
# df = df.append(df_3)
# df = df.append(df_4)

# Ajout d'index au dataset
df.index = np.arange(0,len(df)) 
# np.arange : Retourne des valeurs uniformément espacées dans un intervalle donné

print('Format du jeu de données: {}'.format(df.shape))
print('Exemples :')
print(df.head(10))

print(datetime.now().time())

22:20:38.027892
Format du jeu de données: (24058263, 2)
Exemples :
   Cust_Id  Rating
0       1:     NaN
1  1488844     3.0
2   822109     5.0
3   885013     4.0
4    30878     4.0
5   823519     3.0
6   893988     3.0
7   124105     4.0
8  1248029     3.0
9  1842128     4.0
22:21:00.182836


In [3]:

"""
Réorganisation des données : Ajout des Movie_Id au dataset

temps pour 1 dataset : 6 min
"""

print(datetime.now().time())


# Création d'un dataFrame avec Un Booleen dans la colonne rating
df_without_null = pd.DataFrame(pd.isnull(df.Rating))
# Supprime toutes les valeurs False
df_without_null = df_without_null[df_without_null['Rating'] == True]
# Ajout de nouveaux index
df_without_null = df_without_null.reset_index()


movie_table = []
movie_id = 1


for i,j in zip(df_without_null['index'][1:],df_without_null['index'][:-1]):
    # Création d'une ligne avec une seule valeur : l'ID du film. Il apparait autant de fois que le film a d'avis
    movies_row = np.full((1,i-j-1), movie_id)
    # On le rajoute dans un tableau général
    movie_table = np.append(movie_table, movies_row)
    movie_id += 1

# Manipulation pour le dernier film
last_movie = np.full((1,len(df) - df_without_null.iloc[-1, 0] - 1),movie_id)
movie_table = np.append(movie_table, last_movie)

print('Tableau des ID de film: {}'.format(movie_table))
print('Taille: {}'.format(len(movie_table)))

print(datetime.now().time())

22:21:09.742965
Tableau des ID de film: [1.000e+00 1.000e+00 1.000e+00 ... 4.499e+03 4.499e+03 4.499e+03]
Taille: 24053764
22:26:46.747793


In [5]:
print(datetime.now().time())

"""
Réorganisation des données : Ajout au dataFrame

temps pour 1 dataset : 10 secondes
"""

# On ajoute ce tableau à notre dataFrame
df = df[pd.notnull(df['Rating'])]
df['Movie_Id'] = movie_table.astype(int)
df['Cust_Id'] = df['Cust_Id'].astype(int)

print('Format du jeu de données: {}'.format(df.shape))
print('Exemples')
print(df.head(10))

print(datetime.now().time())

22:29:14.907553
Format du jeu de données: (24053764, 3)
Exemples
    Cust_Id  Rating  Movie_Id
1   1488844     3.0         1
2    822109     5.0         1
3    885013     4.0         1
4     30878     4.0         1
5    823519     3.0         1
6    893988     3.0         1
7    124105     4.0         1
8   1248029     3.0         1
9   1842128     4.0         1
10  2238063     3.0         1
22:29:24.044960


In [6]:
print(datetime.now().time())

"""
Suppression des données "inutiles" : 
- les films avec pas assez de notes : 1700
- les utilisateurs qui de donnent pas assez de notes : 50

temps pour 1 dataset : 15 secondes
"""
movie_min = 1700
cust_min = 50

f = ['count','mean']


# On regroupe les notes par film et on attribue à ce groupe un compte et une moyenne
df_movie_summary = df.groupby('Movie_Id')['Rating'].agg(f)
df_movie_summary.index = df_movie_summary.index.map(int)

# Fait une liste de tous les films dont le nombre de reviews est inférieur à movie_min
drop_movie_list = df_movie_summary[df_movie_summary['count'] < movie_min].index

print('Nombre de films en dessous de la limite : {}'.format(len(drop_movie_list)))

# Même procédé que pour les films
df_cust_summary = df.groupby('Cust_Id')['Rating'].agg(f)
df_cust_summary.index = df_cust_summary.index.map(int)
drop_cust_list = df_cust_summary[df_cust_summary['count'] < cust_min].index

print('Nombre d utilisateurs en dessous de la limite: {}'.format(len(drop_cust_list)))

print(datetime.now().time())

22:30:01.078891
Nombre de films en dessous de la limite : 3092
Nombre d utilisateurs en dessous de la limite: 322675
22:30:10.081165


In [8]:
print(datetime.now().time())

"""
Suppression des données "inutiles" : 
- les films avec pas assez de notes
- les utilisateurs qui de donnent pas assez de notes

temps pour 1 dataset : 20 secondes
"""

print('Taille d origine: {}'.format(df.shape))
# Suppression des films 
df = df[~df['Movie_Id'].isin(drop_movie_list)]
# Suppression des utlisateurs
df = df[~df['Cust_Id'].isin(drop_cust_list)]
print('Taille après la suppression: {}'.format(df.shape))


print(datetime.now().time())

22:31:08.695871
Taille d origine: (17642528, 3)
Taille après la suppression: (17642528, 3)
22:31:21.708305


In [9]:
print(datetime.now().time())

"""
Mapping des données : création d'un dataFrame depuis le fichier movie_titles.csv

temps pour 1 dataset : 0 seconde
"""

df_title = pd.read_csv('./data/movie_titles.csv', encoding = "ISO-8859-1", header = None, names = ['Movie_Id', 'Year', 'Name'])
# Inplace = True : ne crée pas un nouveau dataFrame, le modifie
df_title.set_index('Movie_Id', inplace = True)
# Récupération des dix premiers films
print (df_title.head(10))

print(datetime.now().time())

22:31:40.031264
            Year                          Name
Movie_Id                                      
1         2003.0               Dinosaur Planet
2         2004.0    Isle of Man TT 2004 Review
3         1997.0                     Character
4         1994.0  Paula Abdul's Get Up & Dance
5         2004.0      The Rise and Fall of ECW
6         1997.0                          Sick
7         1992.0                         8 Man
8         2004.0    What the #$*! Do We Know!?
9         1991.0      Class of Nuke 'Em High 2
10        2001.0                       Fighter
22:31:40.136678


In [10]:
#print(datetime.now().time())

"""
Toutes les données dans une matrice

temps pour 1 dataset : 50 secondes
"""

df_matrix = pd.pivot_table(df,values='Rating',index='Cust_Id',columns='Movie_Id')
print("Taille de la matrice des données")
print(df_matrix.shape)
#print(datetime.now().time())

Taille de la matrice des données
(148083, 1407)


In [13]:

print(datetime.now().time())

"""
R de Pearson : Les films similaire à "Lilo et Stitch"

temps pour 1 dataset : 1 minute
"""

min_count=0
movie_title= "Lilo and Stitch"


print("Pour le film {}".format(movie_title))


# Récupération de l'index du film
i = int(df_title.index[df_title['Name'] == movie_title][0])

# Récupération des utilisateurs et de leur note pour ce film
movie_target = df_matrix[i]

# Corrélation avec toute la matrice
similar_to_target = df_matrix.corrwith(movie_target, method='pearson')


# Création d'un dataFrame PearsonR
df_corr_target = pd.DataFrame(similar_to_target, columns = ['PearsonR'])

# Supprimer les valeurs manquantes
df_corr_target.dropna(inplace = True) 

# trie par le dataFrame par PearsonR descendant
df_corr_target = df_corr_target.sort_values('PearsonR', ascending = False)
df_corr_target.index = df_corr_target.index.map(int)

# Rejoindre les dataFrames PearsonR, Title et Movie_summary
df_corr_target = df_corr_target.join(df_title).join(df_movie_summary)[['PearsonR', 'Name', 'count', 'mean']]

# Affiche les 10 premiers PearsonR
print("- Top 10 des films recommandés (basé sur le coefficient de Pearson) - ")
print(df_corr_target[df_corr_target['count']>min_count][:10].to_string(index=False))

print(datetime.now().time())


22:33:53.566004
Pour le film Lilo and Stitch
- Top 10 des films recommandés (basé sur le coefficient de Pearson) - 
 PearsonR                                          Name  count      mean
 1.000000                               Lilo and Stitch  39752  3.823254
 0.400310                                      Gorgeous   1705  3.187097
 0.393666                   The Hunchback of Notre Dame   1883  3.482209
 0.391005                                     Dinotopia   1938  3.327657
 0.386438                                      Hercules  18464  3.680730
 0.374698  Brother Bear (Theatrical Widescreen Version)  33708  3.746707
 0.373512                      The Emperor's New Groove  28955  3.848040
 0.371155                            Air Bud: World Pup   3493  3.127684
 0.364773                                     RoboCop 3   1796  3.073497
 0.363356                          Good Guys Wear Black   1987  2.710116
22:34:12.041541


In [17]:
print(datetime.now().time())

"""
Collaborative Filtering avec SVD : Evalutation de l'algorithme SVD

temps pour 100k: 1 heure 7 minutes
"""

reader = Reader()

data = Dataset.load_from_df(df[['Cust_Id', 'Movie_Id', 'Rating']][:10000], reader)


algorithm_SVD = SVD()
# Validation croisée
cross_validate(algorithm_SVD, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

print(datetime.now().time())


15:45:35.528260
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2568  1.2622  1.2332  1.2566  1.2552  1.2528  0.0101  
MAE (testset)     1.0437  1.0520  1.0154  1.0488  1.0298  1.0379  0.0136  
Fit time          1.17    1.10    1.22    1.13    1.79    1.28    0.26    
Test time         0.04    0.03    0.02    0.03    0.04    0.03    0.01    
15:45:43.610097


In [14]:
print(datetime.now().time())

"""
Collaborative Filtering : Les films aimés (note de 5) par l'utilisateur 785314

temps pour 1 dataset : 0 seconde
"""

# Récupération des films préférés de l'utilisateur 785314
df_user = df[(df['Cust_Id'] == 785314) & (df['Rating'] == 5)]
df_user = df_user.set_index('Movie_Id')
df_user = df_user.join(df_title)['Name']

print(datetime.now().time())

22:34:36.144084
22:34:36.367283


In [19]:
print(datetime.now().time())

"""
Collaborative Filtering : Les films que pourrait aimer l'utilisateur 785314

temps pour 1 dataset : 40 minutes
"""

user_est_movies = df_title.copy()
user_est_movies = user_est_movies.reset_index()
# Suppresion des films qui n'ont pas assez d'avis
user_est_movies = user_est_movies[~user_est_movies['Movie_Id'].isin(drop_movie_list)]


# Récupération de tout le dataset
data = Dataset.load_from_df(df[['Cust_Id', 'Movie_Id', 'Rating']], reader)

trainset = data.build_full_trainset()
algorithm_SVD.fit(trainset)

user_est_movies['Estimate_Score'] = user_est_movies['Movie_Id'].apply(lambda x: algorithm_SVD.predict(785314, x).est)

user_est_movies = user_est_movies.drop('Movie_Id', axis = 1)

user_est_movies = user_est_movies.sort_values('Estimate_Score', ascending=False)
print(user_est_movies.head(10))
print(datetime.now().time())

16:14:05.633989
        Year                                        Name  Estimate_Score
4391  1993.0                            Army of Darkness        5.000000
3167  1987.0                   Evil Dead 2: Dead by Dawn        5.000000
2318  2003.0  The Looney Tunes Golden Collection: Vol. 1        4.807591
1019  1989.0                      The Simpsons: Season 1        4.803316
3768  2004.0            Wonderfalls: The Complete Series        4.792415
4382  2004.0              Farscape: The Peacekeeper Wars        4.788926
2194  2004.0  The Looney Tunes Golden Collection: Vol. 2        4.759907
2113  2002.0                                     Firefly        4.721353
3443  2004.0       Family Guy: Freakin' Sweet Collection        4.708142
1799  2002.0                 An Evening With Kevin Smith        4.689776
16:58:00.465138
